# Do not run, I imported nothing

In [ ]:
poke_df[['HP', 'Attack', 'Defense']].describe()
# always describe

## Binarization

In [ ]:
# Did they listen to the song, once or more?
watched = np.array(popsong_df['listen_count']) # make column array
watched[watched >= 1] = 1 # MADE ALL VALUES OVER 2, EQUAL TO 1. Now 0's and 1's!
popsong_df['watched'] = watched

In [4]:
import numpy as np
thearray = np.array([1,2,3,4,5,6,7,8,9])
thearray[thearray >= 4] = 1 # MADE ALL VALUES OVER 4, EQUAL TO 1
thearray

array([1, 2, 3, 1, 1, 1, 1, 1, 1])

In [ ]:
# all 0's and 1's
from sklearn.preprocessing import Binarizer # SEARCH UP BINARIZER
bn = Binarizer(threshold=0.9)
pd_watched = bn.transform([popsong_df['listen_count']])[0]
popsong_df['pd_watched'] = pd_watched
popsong_df.head(11)

In [ ]:
# ROUNDING
items_popularity = pd.read_csv('datasets/item_popularity.csv',  
                               encoding='utf-8')
items_popularity['popularity_scale_10'] = np.array(
                   np.round((items_popularity['pop_percent'] * 10)),  
                   dtype='int')
items_popularity['popularity_scale_100'] = np.array(
                  np.round((items_popularity['pop_percent'] * 100)),    
                  dtype='int')

In [ ]:
# INTERACTIONS
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(degree=2, interaction_only=False,  
                        include_bias=False)
res = pf.fit_transform(atk_def)

pd.DataFrame(pf.powers_, columns=['Attack_degree',  
                                  'Defense_degree']) # This turned pokemon attack/Defense into 0-1-2

## Binning

In [ ]:
fcc_survey_df['Age_bin_round'] = np.array(np.floor(
                              np.array(fcc_survey_df['Age']) / 10.))
fcc_survey_df[['ID.x', 'Age', 'Age_bin_round']].iloc[1071:1076]

# 17 becomes 1, 22 becomes 2... that's a bin round.

In [ ]:
bin_ranges = [0, 15, 30, 45, 60, 75, 100]
bin_names = [1, 2, 3, 4, 5, 6]
fcc_survey_df['Age_bin_custom_range'] = pd.cut(        # make a new column
                                           np.array(
                                              fcc_survey_df['Age']), 
                                              bins=bin_ranges) # shows [15,30][30,45] in the rows.
fcc_survey_df['Age_bin_custom_label'] = pd.cut(        # make a new column
                                           np.array(
                                              fcc_survey_df['Age']), 
                                              bins=bin_ranges, # shows 1,2,2,4,5,3 in the rows.         
                                              labels=bin_names)
# view the binned features 
fcc_survey_df[['ID.x', 'Age', 'Age_bin_round', 
               'Age_bin_custom_range',   
               'Age_bin_custom_label']].iloc[1071:1076] # took specific rows sample to show

In [ ]:
# ADAPTIVE BINNING
quantile_list = [0, .25, .5, .75, 1.]
quantiles = fcc_survey_df['Income'].quantile(quantile_list) # lowest to highest stored in quantiles
quantiles

Output
------
0.00      6000.0
0.25     20000.0
0.50     37000.0
0.75     60000.0
1.00    200000.0
Name: Income, dtype: float64

In [ ]:
# HISTOGRAM SALARY, BUT BINS SHOW UP AS RED LINES ! Nice visual to show percentage bins
fig, ax = plt.subplots()
fcc_survey_df['Income'].hist(bins=30, color='#A9C5D3', 
                             edgecolor='black', grid=False)
for quantile in quantiles:
    qvl = plt.axvline(quantile, color='r')
ax.legend([qvl], ['Quantiles'], fontsize=10)
ax.set_title('Developer Income Histogram with Quantiles', 
             fontsize=12)
ax.set_xlabel('Developer Income', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)

In [ ]:
quantile_labels = ['0-25Q', '25-50Q', '50-75Q', '75-100Q']
fcc_survey_df['Income_quantile_range'] = pd.qcut(
                                            fcc_survey_df['Income'], 
                                            q=quantile_list) # column shows [5999,20000][37000,60000] etc
fcc_survey_df['Income_quantile_label'] = pd.qcut(
                                            fcc_survey_df['Income'], 
                                            q=quantile_list,       
                                            labels=quantile_labels) # column shows 0-25Q,50-75Q,25-50Q etc.

fcc_survey_df[['ID.x', 'Age', 'Income', 'Income_quantile_range', 
               'Income_quantile_label']].iloc[4:9]

In [ ]:
fcc_survey_df['Income_log'] = np.log((1+ fcc_survey_df['Income']))
fcc_survey_df[['ID.x', 'Age', 'Income', 'Income_log']].iloc[4:9]
# log makes skewed distribution as normal-like as possible.


## box com transform

In [ ]:
income = np.array(fcc_survey_df['Income']) # make column np array
income_clean = income[~np.isnan(income)] # get rid of isnans
l, opt_lambda = spstats.boxcox(income_clean) # Get optimal lambda value
print('Optimal lambda value:', opt_lambda)

Output
------
Optimal lambda value: 0.117991239456

In [ ]:
fcc_survey_df['Income_boxcox_lambda_0'] = spstats.boxcox(
                                        (1+fcc_survey_df['Income']), 
                                          lmbda=0) # normal-like income values
fcc_survey_df['Income_boxcox_lambda_opt'] = spstats.boxcox(
                                            fcc_survey_df['Income'], 
                                              lmbda=opt_lambda) # other normal-like lambda values

fcc_survey_df[['ID.x', 'Age', 'Income', 'Income_log', 
               'Income_boxcox_lambda_0',       
               'Income_boxcox_lambda_opt']].iloc[4:9]